In [1]:
import sys
sys.path.extend(['../', '../read_utils/'])

import json
import numpy as np
from pathlib import Path

from read_utils.main import (
    calibrate_magnetic_wifi_ibeacon_to_position, 
    extract_ibeacon_rssi, 
    visualize_heatmap, 
    save_figure_to_html,
)


In [2]:
floor_data_dir = '../data/site1/F1'
path_data_dir = floor_data_dir + '/path_data_files'
floor_plan_filename = floor_data_dir + '/floor_image.png'
floor_info_filename = floor_data_dir + '/floor_info.json'

save_dir = '../output/site1/F1'
ibeacon_image_save_dir = save_dir + '/ibeacon_images'

In [3]:
with open(floor_info_filename) as f:
    floor_info = json.load(f)
width_meter = floor_info["map_info"]["width"]
height_meter = floor_info["map_info"]["height"]
print(width_meter, height_meter)

path_filenames = list(Path(path_data_dir).resolve().glob("*.txt"))
mwi_datas = calibrate_magnetic_wifi_ibeacon_to_position(path_filenames)
step_positions = np.array(list(mwi_datas.keys()))
print(step_positions.shape)

239.81749314504376 176.44116534000818
(7840, 2)


In [4]:
ibeacon_rssi = extract_ibeacon_rssi(mwi_datas)
print(len(ibeacon_rssi))
ten_ibeacon_ummids = list(ibeacon_rssi.keys())[0:10]
print(ten_ibeacon_ummids)

20
['9195B3AD-A9D0-4500-85FF-9FB0F65A5201_0_0', 'FDA50693-A4E2-4FB1-AFCF-C6EB07647825_10073_61418', 'FB349B5F-8000-0080-0010-00003CFE0000_6155_59255', '6B76E28A-6FA2-48C9-8502-C1DAA388AB2C_47237_15305', '6B76E28A-6FA2-48C9-8502-C1DAA388AB2C_45850_14031', '6B76E28A-6FA2-48C9-8502-C1DAA388AB2C_19018_35270', '616C6970-6179-626F-7869-626561636F6A_49662_37272', '3C3493F1-997A-D187-0AD6-FB5A8D4EFA63_30242_6415', '616C6970-6179-626F-7869-626561636F6A_3824_27065', '616C6970-6179-626F-7869-626561636F6A_28214_49911']


In [5]:
target_ibeacon = '9195B3AD-A9D0-4500-85FF-9FB0F65A5201_0_0'
heat_positions = np.array(list(ibeacon_rssi[target_ibeacon].keys()))
heat_values = np.array(list(ibeacon_rssi[target_ibeacon].values()))[:, 0]
print(heat_positions.shape, heat_values.shape)

fig = visualize_heatmap(
    heat_positions, 
    heat_values, 
    floor_plan_filename, 
    width_meter, 
    height_meter, 
    colorbar_title='dBm', 
    title=f'iBeacon: {target_ibeacon} RSSI', 
    show=False,
)
html_filename = f'{ibeacon_image_save_dir}/{target_ibeacon}.html'
html_filename = str(Path(html_filename).resolve())
save_figure_to_html(fig, html_filename)

(6003, 2) (6003,)


In [6]:
# Identify the missing positions
missing_positions_mask = np.isin(step_positions, heat_positions).all(axis=1)

# Extract the missing positions
missing_positions = step_positions[~missing_positions_mask]
missing_positions.shape

(1837, 2)

## Method 3: Inverse Distance Weighting (IDW) Interpolation

In [7]:
def inverse_distance_weighting(known_points, known_values, unknown_points, power=2):
    """
    Interpolate using Inverse Distance Weighting.
    """
    interpolated_values = []
    for point in unknown_points:
        # Compute distances to known_points
        distances = np.linalg.norm(known_points - point, axis=1)
        # Avoid divide by zero
        distances[distances == 0] = 1e-5
        # Compute weights
        weights = 1.0 / distances**power
        # Compute weighted average
        value = np.sum(weights * known_values) / np.sum(weights)
        interpolated_values.append(value)
    return np.array(interpolated_values)

# Interpolate using IDW
idw_interpolated_values = inverse_distance_weighting(heat_positions, heat_values, missing_positions)
idw_interpolated_values.shape

(1837,)

In [8]:
all_positions = np.vstack([heat_positions, missing_positions])
all_idw_values = np.hstack([heat_values, idw_interpolated_values])

fig = visualize_heatmap(
    heat_positions, 
    heat_values, 
    floor_plan_filename, 
    width_meter, 
    height_meter, 
    colorbar_title='dBm', 
    title=f'iBeacon: {target_ibeacon} RSSI', 
    show=False,
)
html_filename = f'{ibeacon_image_save_dir}/{target_ibeacon}-idw.html'
html_filename = str(Path(html_filename).resolve())
save_figure_to_html(fig, html_filename)